In [1]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            f"Argument {name} must be {bound_types[name]}"
                        )
            return func(*args, **kwargs)

        return wrapper
    return decorate


In [4]:
@typeassert(int, int)
def add(x, y):
    return x + y

print(add(2, 3))  # Outputs: 5

try:
    print(add(2, 'hello'))  # Raises TypeError: Argument y must be <class 'int'>
except TypeError as e:
    print(e)

@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

spam(1, 2, 3)  # Outputs: 1 2 3
spam(1, 'hello', 3)  # Outputs: 1 hello 3

try:
    spam(1, 'hello', 'world')  # Raises TypeError: Argument z must be <class 'int'>
except TypeError as e:
    print(e)


5
Argument y must be <class 'int'>
1 2 3
1 hello 3
Argument z must be <class 'int'>
